In [1]:
import os
import sys
# import api, output, auth_api
from other_modules import output,caption
from download_youtube_code import api,auth_api
# import caption
from datetime import datetime
import pandas as pd

In [2]:
!pwd

/Users/evelyn/Desktop/ConceptMap/ConceptGuide_code-main/MyResearchCode


In [3]:
service = auth_api.get_authenticated_service()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=930864150966-jne9djg1btv397qfr6hctjreite2fqq5.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=jVsXeesL53c3VE8qH6R48Z2ievyLTh&prompt=consent&access_type=offline


Enter the authorization code:  4/1AX4XfWi-8hjpE_BAC-w2UFOcqKd_QGUg-GWUcWDkwXhC7WAJUgUsvvUUieo


In [4]:
# ##### Comment Threads ######
# # refer to: https://developers.google.com/youtube/v3/docs/commentThreads/list
# # get comment's information
# # 1.videoid
# # 2.textDisplay (textOriginal)
# # 3.publishedAt
# # 4.authorDisplayName
# # 5.authorProfileImageUrl
# def get_comment_threads(youtube, vlist):
#     comments = []
#     for video in vlist:
#         vid = video['videoid']
#         #print("vid = ",vid)
#         try:
#             results = youtube.commentThreads().list(
#                 part="snippet",#replies
#                 videoId=vid,
#                 maxResults=100, # at most 100 comments
#                 textFormat="plainText"## or 'html'
#             ).execute()
#             for item in results["items"]:
#                 if 'snippet' in item:
#                     data = {
#                         'videoId':item['snippet']['videoId'],
#                         'textDisplay':item['snippet']['topLevelComment']['snippet']['textDisplay'],
#                         'publishedAt':item['snippet']['topLevelComment']['snippet']['publishedAt'],
#                         'authorDisplayName':item['snippet']['topLevelComment']['snippet']['authorDisplayName'],#
#                         'authorProfileImageUrl':item['snippet']['topLevelComment']['snippet']['authorProfileImageUrl']#
#                     }
#                     comments.append(data)
#         except:
#             print("videoid: ",vid," do not have comments!")
#     return comments

In [4]:
##### Comment Threads ######
# refer to: https://developers.google.com/youtube/v3/docs/commentThreads/list
# get comment's information
# 1.videoid
# 2.textDisplay (textOriginal)
# 3.publishedAt
# 4.authorDisplayName
# 5.authorProfileImageUrl
gl = []
def get_comment_threads(youtube, nextPageToken, vid, comments):
    # for video in vlist:
    #     vid = video['videoid']
    #     #print("vid = ",vid)
    try:
        results = youtube.commentThreads().list(
            part="id, snippet, replies",#replies
            videoId=vid,
            maxResults=100, # at most 100 comments
            textFormat="plainText" ## or 'html'
            # pageToken = nextPageToken
        ).execute()
        while results: 
            for item in results["items"]:
                if 'snippet' in item:
                    data = {
                        'commentId':item['snippet']['topLevelComment']['id'],
                        'parentId':'0',
                        'videoId':item['snippet']['videoId'],
                        'textDisplay':item['snippet']['topLevelComment']['snippet']['textDisplay'],
                        'publishedAt':item['snippet']['topLevelComment']['snippet']['publishedAt'],
                        'authorDisplayName':item['snippet']['topLevelComment']['snippet']['authorDisplayName'],#
                        'authorProfileImageUrl':item['snippet']['topLevelComment']['snippet']['authorProfileImageUrl']#
                    }
                    # print(data)
                    comments.append(data)
                    # replies = item.get('replies')
                    if item["snippet"]['totalReplyCount'] > 0 and 'replies' in item.keys():
                        tmp = []
                        # reply would be stored backwards
                        for reply in item['replies']['comments']:
                            data = {
                                'commentId':reply['id'],
                                'parentId':reply['snippet']['parentId'],
                                'videoId':reply['snippet']['videoId'],
                                'textDisplay':reply['snippet']['textDisplay'],
                                'publishedAt':reply['snippet']['publishedAt'],
                                'authorDisplayName':reply['snippet']['authorDisplayName'],#
                                'authorProfileImageUrl':reply['snippet']['authorProfileImageUrl']#
                            }
                            tmp.append(data)
                            # Extract reply
                        tmp_rev = []
                        tmp_rev.append(tmp[::-1])
                        for i in tmp_rev:
                            for j in i:
                                # print(j)
                                comments.append(j)
                            # reply = reply['snippet']['textDisplay']
                            # print(reply)
                            # Store reply is list
                            # replies.append(reply)
            if 'nextPageToken' in results:
                results = youtube.commentThreads().list(
                    part="id, snippet, replies",#replies
                    videoId=vid,
                    maxResults=100, # at most 100 comments
                    textFormat="plainText", ## or 'html'
                    pageToken = results['nextPageToken']
                ).execute()
            else:
                break
        return comments
        # extracting required info
        # from each result object 
        # return results
    except:
        print("videoid: ",vid," do not have comments!")
# return comments
# def load_comments(results, comments):
#     for item in results["items"]:
#         if 'snippet' in item:
#             data = {
#                 'videoId':item['snippet']['videoId'],
#                 'textDisplay':item['snippet']['topLevelComment']['snippet']['textDisplay'],
#                 'publishedAt':item['snippet']['topLevelComment']['snippet']['publishedAt'],
#                 'authorDisplayName':item['snippet']['topLevelComment']['snippet']['authorDisplayName'],#
#                 'authorProfileImageUrl':item['snippet']['topLevelComment']['snippet']['authorProfileImageUrl']#
#             }
#             # print(data)
#             comments.append(data)
            

In [5]:
# vlist = ['48Gxwmofaho', 'dIUTsFT2MeQ', 'L1ung0wil9Y', 'nKW8Ndu7Mjw',
#          'cKxRvEZd3Mw', 'OQ5jsbhAv_M', '3w_ozqJNLvA', 'EH6vE97qIP4',
#          'X8jsijhllIA', 'DMtFhACPnTY']
# nlist = ['coco_skeleton_siggraph', 'nlp_with_spacy', 'assembly', 'ml_7steps_google',
#          'ml_recipes_google', 'dp_mit', 'vr_martial_arts_ieee', 'blockchain_mit',
#          'hamming_code', 'sha256']

# #nlp_comments
# vlist = ['fOvTtapxa9c', '5ctbvkAMQO4', 'd4gGtcobq8M', 'IpBZ01g0pGE', 'FLZvOKSCkxY', 
#          'AJVP96tAWxw', '3Q8wacwA4gs', '6WpnxmmkYys', 'MNvT5JekDpg', 'Sx3Fpw0XCXk',
#          '05ONoGfmKvA', 'bDPULOFFlaI', 'DDByc9LyMV8', 'FggN0VtWYTk', 'pHiIwQk57dc',
#          'BXpmNX_1Dy8', 'mhHfnhh-pB4', 'yGKTphqxR9Q', 'X2vAabgKiuM', 'J5IlKj7H8T8',
#          'ReakZVh2Xwk', 'imPpT2Qo2sk', 'ohM7D21C_8Q', 'iGmHnICXDss']

# #covid_comments
vlist = ['5DGwOJXSxqg', 'ha2mLz-Xdpg', 'vN30emwcNS4', 'muiz5PHYNKA',
         '35Idb_lCU4o']
nlist = ['COVID-19 Animation', 'Vitamin D and COVID 19', 'If You Get COVID 19', 'Bulgaria protesters',
         'COVID-19 Vaccines']

In [6]:
filename_root = '/Users/evelyn/Desktop/ConceptMap/experimental/to_label/covid/'
# file_specific = 'hamming_code'
## get videos comments ##
# csv_files = glob.glob(os.path.join(path, "*.csv"))
# videos_ = []
for i in range(len(vlist)):
    comments = []
    videos_3 = get_comment_threads(service, "", vlist[i], comments)
    # if type(videos_3) == list:
    #     for i in videos_3:
    #         videos_.append(i)
    # print('')
    output.to_csv(videos_3, filename_root+nlist[i]+"_comment.csv")

In [7]:
# print(len(videos_))

In [77]:
# output.to_csv(videos_, "/Users/evelyn/Desktop/ConceptMap/experimental/raw/nlp_comment.csv")

In [60]:
lst=["a","b","c","d","e","f"]
lst_rev=[]
lst_rev.append(lst[::-1])
for i in lst_rev:
    for j in i:
        print (j)
print(lst_rev)

f
e
d
c
b
a
[['f', 'e', 'd', 'c', 'b', 'a']]
